In [2]:
#Using Overview for Webscraping with OpenAI API from here https://medium.com/@joelpantoja/web-scraping-with-openai-6ceace410191 

In [3]:
import requests
from bs4 import BeautifulSoup
import tiktoken
from requests import Response
from openai import OpenAI, ChatCompletion
from dotenv import load_dotenv
import os

from pprint import pprint

In [4]:
#Load environment and API Key
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")


In [21]:
import undetected_chromedriver as uc
from bs4 import BeautifulSoup
import time

def get_real_jooble_html(url):
    # This launches a version of Chrome that Cloudflare usually trusts
    driver = uc.Chrome(headless=False) # Start with False to see if it works
    
    driver.get(url)
    
    # Wait for the "Just a moment" to disappear (usually 5-10 seconds)
    time.sleep(10) 
    
    # Grab the HTML AFTER the challenge is solved
    html = driver.page_source
    driver.quit()
    
    return html

url = 'https://jooble.org/SearchResult?rgns=Remote&ukw=data%20scientist'
raw_html = get_real_jooble_html(url)

# Now you can pass raw_html to your BeautifulSoup function
soup = BeautifulSoup(raw_html, 'html.parser')

In [ ]:
def num_tokens_from_string(string: str, encoding_name: str = "gpt-3.5-turbo") -> int:
    encoding = tiktoken.encoding_for_model(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens


def chat_completion_request(messages, tools=None, tool_choice=None, model="gpt-3.5-turbo"):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
            response_format={'type':'json_object'}
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        return e

In [ ]:
# Set OpenAI Format
tools = [
    {
        "type": "function",
        "function": {
            "name": "product_catalog_overview",
            "description": "Retrieves a high level overview of the books in a book catalog page",
            "parameters": {
                "type": "object",
                "properties": {
                    "title": {
                        "type": "string",
                        "description": "The title of the book",
                    },
                    "price": {
                        "type": "string",
                        "description": "The price of the book in pounds",
                    },
                    "rating": {
                        "type": "integer",
                        "description": "The rating of the book. This should be displayed as a number of stars",
                    },
                    "in_stock": {
                        "type": "boolean",
                        "description": "Whether the book is in stock. The value should be true or false",
                    }
                },
                "required": ["title", "price", "rating"]
            },
        }
    },
]